# This notebook creates the dataframe to look at the z-score from a temporal perspective

### What we have
- We already have the dataframe of the z-scores for every unit. This doesn't tell us much about the time though
- we have old spike collection that gives us the neurons spikes across time and the event dicts and their times

### What we'll do
- First, break up these events by time they happened, for example 1/3's where we just create the z-score dataframe for the events that fall into the first 3rd of the recording time, then the 2/3, then 3/3 we then plot the z-scored during these 3 time periods seperately for events to see if it changes
- second, use gaussian smoothing filter to see individual neurons continuously changing 


In [ ]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as collection
import spike.spike_analysis.zscoring as zscoring
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as scipy
from scipy.ndimage import gaussian_filter1d
from scipy.signal import fftconvolve
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import pickle
import re

In [3]:
pd.set_option('display.max_colwidth', 0)  # 0 means unlimited in newer pandas versions

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Don’t truncate column contents
pd.set_option("display.max_colwidth", None)

# Expand the display to the full width of the screen
pd.set_option("display.width", 0)


In [4]:
spike_collection_json_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\spike_collection.json'

In [5]:
sp = sc.SpikeCollection.load_collection(spike_collection_json_path)

#### Quick look at the events

In [6]:
rec_events = sp.recordings[0].event_dict

# get unique event names from rec_events dictionary
event_names = list(rec_events.keys())
print("Unique event names:", event_names)

Unique event names: ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']


#### Quick Look at all the recording names

In [7]:
# printing unique recording names
recording_names = [rec.name for rec in sp.recordings]
print("Unique recording names:", recording_names)

Unique recording names: ['20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec', '20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.rec', '20230618_100636_standard_c

In [12]:
# list of recording objects in sp
recs = sp.recordings
print(recs)


[<spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E094280>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E076AD0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7CD35720>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E0764D0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E03E9B0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E03C9A0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F021E1C00>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E076620>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7E075AE0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F7CD375E0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F021E17E0>, <spike.spike_analysis.spike_recording.SpikeRecording object at 0x0000023F02

In [19]:
print(recs[0].unit_array[:20])


[210  65  74  73  85  85 196 210  51 211  73 192  65 211 196 126  22  85
  34  65]


In [ ]:
def gaussian_kernel_bins(sigma_ms: float, timebin_ms: float, support: float = 3.0) -> np.ndarray:
    """
    Discrete symmetric Gaussian in *bins*, normalized so that
    convolving counts/bin yields *Hz* directly.
    """
    sigma_bins = float(sigma_ms) / float(timebin_ms)
    half = int(np.ceil(support * sigma_bins))
    n = np.arange(-half, half + 1, dtype=float)
    k = np.exp(-0.5 * (n / sigma_bins) ** 2)

    dt = timebin_ms / 1000.0  # seconds per bin
    k /= (k.sum() * dt)       # ensures output is in Hz
    return k


def smooth_rate_fft(train_counts: np.ndarray, kernel_bins: np.ndarray) -> np.ndarray:
    pad = len(kernel_bins) // 2
    x = np.pad(train_counts.astype(float), pad, mode='reflect')
    rate_hz = fftconvolve(x, kernel_bins, mode='same')[pad:-pad]
    return rate_hz

def smooth_rate_ndimage(train_counts: np.ndarray, sigma_ms: float, timebin_ms: float) -> np.ndarray:
    """
    Convenience: gaussian_filter1d returns counts/bin; divide by dt to get Hz.
    """
    sigma_bins = float(sigma_ms) / float(timebin_ms)
    counts_sm = gaussian_filter1d(train_counts.astype(float),
                                  sigma=sigma_bins, mode='reflect', truncate=3.0)
    dt = timebin_ms / 1000.0
    return counts_sm / dt


def zscore_from_baseline(rate_hz: np.ndarray, t_rel_ms: np.ndarray,
                         baseline_ms=(-1500.0, -250.0), robust=False) -> np.ndarray:
    b = (t_rel_ms >= baseline_ms[0]) & (t_rel_ms <= baseline_ms[1])
    bvals = rate_hz[b]
    mu = np.mean(bvals) if bvals.size else 0.0
    if robust and bvals.size:
        med = np.median(bvals)
        mad = np.median(np.abs(bvals - med))
        sd = 1.4826 * mad if mad > 0 else np.std(bvals, ddof=1)
    else:
        sd = np.std(bvals, ddof=1) if bvals.size else 0.0
    sd = sd if sd > 1e-12 else 1e-12
    return (rate_hz - mu) / sd



In [ ]:
'''
Inputs:
One part we're specifying here the input values we want for the gaussian kernel aka
 - sigma_ms, our standard deviation in ms
 - timebin_ms, the size of our bins in ms


'''

def compute_unit_ztrials_ms(rec, unit_id, event_windows_ms,
                            win_ms=(-2000.0, 8000.0),
                            timebin_ms=5.0,
                            sigma_ms=75.0,
                            baseline_ms=(-1500.0, -250.0),
                            use_fft=True, robust=False):
    """
    rec.unit_timestamps[unit_id] must be in SAMPLES (your class format).
    event_windows_ms: array-like of [start_ms, end_ms] pairs (same as rec.event_dict[...] entries).
    Returns: t_rel_ms (T,), z_trials (n_events, T), rate_trials_hz (n_events, T)
    """
    # --- build the common time grid relative to onset
    T = int(np.floor((win_ms[1] - win_ms[0]) / timebin_ms))
    t_rel_ms = np.arange(T) * timebin_ms + win_ms[0]

    # --- choose smoother
    if use_fft:
        k = gaussian_kernel_bins(sigma_ms, timebin_ms, support=3.0)
        smooth = lambda counts: smooth_rate_fft(counts, k)
    else:
        smooth = lambda counts: smooth_rate_ndimage(counts, sigma_ms, timebin_ms)

    # --- spikes for this unit (samples -> ms)
    spikes_samples = rec.unit_timestamps[str(unit_id)] if str(unit_id) in rec.unit_timestamps else rec.unit_timestamps[unit_id]
    spikes_ms = spikes_samples * (1000.0 / rec.sampling_rate)

    z_trials = []
    rate_trials = []

    for (start_ms, _end_ms) in np.asarray(event_windows_ms):
        t0 = start_ms + win_ms[0]
        t1 = start_ms + win_ms[1]
        # make_bin edges
        edges = np.arange(t0, t1 + timebin_ms, timebin_ms)
        counts, _ = np.histogram(spikes_ms, bins=edges)  # counts/bin
        # smooth -> Hz
        rate_hz = smooth(counts)
        # z-score on relative axis
        z = zscore_from_baseline(rate_hz, t_rel_ms, baseline_ms, robust=robust)
        z_trials.append(z)
        rate_trials.append(rate_hz)

    return t_rel_ms, np.vstack(z_trials), np.vstack(rate_trials)
